## Import  libs

In [1]:
import cv2
import torch
import numpy as np
from deep_sort_realtime.deepsort_tracker import DeepSort
from models.common import DetectMultiBackend, AutoShape


c:\Users\ngvul\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Config value

In [2]:
video_path = r"./data_ext/caotoc.mp4"
conf_threshold = 0.6
tracking_class = 2

## Init DeepSort

In [3]:
tracker = DeepSort(max_age=5) #max_age = 5 : after 5s will remove if it doesnt appear

## Load model

In [4]:
device = "cpu"
model = DetectMultiBackend(weights=r".\weights\yolov9-c-converted.pt")
model = AutoShape(model)

Fusing layers... 
gelan-c summary: 387 layers, 25288768 parameters, 23646704 gradients, 102.1 GFLOPs
Adding AutoShape... 


In [16]:
#Load classname from file classname
with open(r".\data_ext\classname.txt") as f:
    class_names = f.read().strip().split('\n')
print(class_names)
color = np.random.randint(0,255, size=(len(class_names),3 ))
print(color.shape)
tracks = []

#Init VideoCapture from CV2:
cap = cv2.VideoCapture(video_path)

#Read frames 
while True:
    ret,frame = cap.read()
    #Dectection through Model
    result = model(frame)
    detect = []
    for detect_object  in result.pred[0]:
        label ,confidence, bbox  = detect_object[5],detect_object[4] , detect_object[:4]
        x1,y1,x2,y2 = map(int,bbox)
        class_id = int(label)
        if tracking_class is None:
            if confidence < conf_threshold:
                continue
            else:
                if class_id == tracking_class or confidence < conf_threshold:
                    continue
        detect.append([[x1,y1,x2-x1,y2-y1],confidence,class_id])
    tracks = tracker.update_tracks(detect,frame=frame)
    for track in tracks:
        if track.is_confirmed():
            track_id = track.track_id
            ltrb = track.to_ltrb()
            class_id = track.get_det_class()
            colorz = color[class_id]
            x1,y1,x2,y2 = map(int,ltrb)
            B , G , R = map(int,colorz)
            label = "{}-{}".format(class_names[class_id],track_id)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (B, G, R), 2)
            cv2.rectangle(frame, (x1 - 1, y1 - 20), (x1 + len(label) * 12, y1), (B, G, R), -1)
            cv2.putText(frame, label, (x1 + 5, y1 - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    #Show img
    cv2.imshow("OT",frame)
    if cv2.waitKey(1) == ord("q"):
        break
cap.release()
cv2.destroyAllWindows()



['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'potted plant', 'bed', 'dining table', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']
(80, 3)
